In [82]:
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [83]:
from imblearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import cross_val_score

In [84]:
ruta='datos/'
df_train_three=joblib.load(ruta+'df_train_datos2.pkl')
df_xtest_three=joblib.load(ruta+'df_xtest_datos2.pkl')



scaler = StandardScaler()

X=df_train_three.loc[:,~df_train_three.columns.isin(['target_duo', 'target_trio'])]
X_real_test=df_xtest_three.loc[:,~df_xtest_three.columns.isin(['target_duo', 'target_trio'])]
y2=df_train_three.loc[:,df_train_three.columns.isin(['target_duo'])]
y3=df_train_three.loc[:,df_train_three.columns.isin(['target_trio'])]

scaler.fit(X)
X=scaler.transform(X)
X_real_test=scaler.transform(X_real_test)

test_size=0.7
random_state=1000
X_train2, X_test2, y_train_2, y_test_2 = train_test_split(X, y2, test_size=test_size, random_state=random_state)
X_train3, X_test3, y_train_3, y_test_3 = train_test_split(X, y3, test_size=test_size, random_state=random_state)

# Set balanceado

Podemos observar que el añadido del oversampler y under apenas tiene efecto. En ambos casos empeora la calidad del resultado.

In [85]:
steps=[('scaler',StandardScaler()),('RFC',RandomForestClassifier(n_estimators=900,n_jobs=-1))]
tubo=Pipeline(steps)

steps2=[('over',RandomOverSampler()),('scaler',StandardScaler()),('RFC',RandomForestClassifier(n_estimators=900,n_jobs=-1))]
tubo2=Pipeline(steps2)

steps3=[('under',RandomUnderSampler()),('scaler',StandardScaler()),('RFC',RandomForestClassifier(n_estimators=900,n_jobs=-1))]
tubo3=Pipeline(steps3)


In [86]:
# Fit pipeline on training data
tubo.fit(X_train2, y_train_2.values.ravel())

# Score pipeline on testing data
tubo.score(X_test2, y_test_2.values.ravel())

0.7911255411255411

In [87]:
# Fit pipeline on training data
tubo2.fit(X_train2, y_train_2.values.ravel())

# Score pipeline on testing data
tubo2.score(X_test2, y_test_2.values.ravel())

0.7905002405002405

In [88]:
# Fit pipeline on training data
tubo3.fit(X_train2, y_train_2.values.ravel())

# Score pipeline on testing data
tubo3.score(X_test2, y_test_2.values.ravel())

0.7869408369408369

# Set no balanceado 

No parece que a priori el balanceo mejore el resultado

In [89]:
# Fit pipeline on training data
tubo.fit(X_train3, y_train_3.values.ravel())

# Score pipeline on testing data
tubo.score(X_test3, y_test_3.values.ravel())

0.773953823953824

In [90]:
# Fit pipeline on training data
tubo2.fit(X_train3, y_train_3.values.ravel())

# Score pipeline on testing data
tubo2.score(X_test3, y_test_3.values.ravel())

0.7648148148148148

In [91]:
# Fit pipeline on training data
tubo3.fit(X_train3, y_train_3.values.ravel())

# Score pipeline on testing data
tubo3.score(X_test3, y_test_3.values.ravel())

0.6464405964405965

In [92]:
# Preparamos salida.

salida_prediccion=tubo.predict(X_real_test)

In [93]:
df_xtest_three["respuesta"]=salida_prediccion
df_xtest_three["status_group"]=df_xtest_three["respuesta"].replace([0, 1,2],['functional', 'functional needs repair','non functional'])


In [94]:
df_xtest_three['status_group'].value_counts()

functional                 9322
non functional             5128
functional needs repair     400
Name: status_group, dtype: int64

In [95]:
df_xtest_three['status_group'].head()

id
50785    non functional
51630        functional
17168    non functional
45559    non functional
49871        functional
Name: status_group, dtype: object

In [96]:
# escribe respuesta
#df_xtest_three['status_group'].to_csv('datos/'+'respuesta2.csv') Su modelo tenía una puntuación de 0.77